In [11]:
pip install fuzzywuzzy

Note: you may need to restart the kernel to use updated packages.


In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process

In [13]:
movie = pd.read_csv(r"D:\Python USB\KNN Movies/movies.csv")
rating = pd.read_csv(r"D:\Python USB\KNN Movies/ratings.csv")

movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
movie['year'] = movie.title.str.extract('(\(\d\d\d\d\))',expand=False)
movie['year'] = movie.year.str.extract('(\d\d\d\d)' ,expand=False)

In [5]:
movie['title'] = movie.title.str.replace('(\(\d\d\d\d\))', '')
movie['title'] = movie['title'].str.strip()
movie.head()

C:\Users\asus\AppData\Local\Temp\ipykernel_17744\3674160478.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  movie['title'] = movie.title.str.replace('(\(\d\d\d\d\))', '')


,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [6]:
rating.groupby('movieId').rating.mean()
rating.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [7]:
df = movie.join(rating,lsuffix='N' , rsuffix='K')
df

,movieIdN,title,genres,year,userId,movieIdK,rating,timestamp
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,1,296,5.0,1147880044
1,2,Jumanji,Adventure|Children|Fantasy,1995,1,306,3.5,1147868817
2,3,Grumpier Old Men,Comedy|Romance,1995,1,307,5.0,1147868828
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995,1,665,5.0,1147878820
4,5,Father of the Bride Part II,Comedy,1995,1,899,3.5,1147868510
...,...,...,...,...,...,...,...,...
62418,209157,We,Drama,2018,494,102903,3.5,1532809067
62419,209159,Window of the Soul,Documentary,2001,494,103042,3.0,1532809256
62420,209163,Bad Poems,Comedy|Drama,2018,494,103249,3.0,1532809116
62421,209169,A Girl Thing,(no genres listed),2001,494,103372,3.0,1532911673


In [8]:
a = 'ab'
b = 'bcd'
print(a+b)

abbcd


In [9]:
df = df.drop(['movieIdK','genres','year','timestamp'], axis=1 )

In [14]:
movie_users = df.pivot(index='movieIdN',columns='userId',values='rating').fillna(0)
matrix_movies_users=csr_matrix(movie_users.values)

In [26]:
matrix_movies_users

<62423x494 sparse matrix of type '<class 'numpy.float64'>'
	with 62423 stored elements in Compressed Sparse Row format>

In [15]:
knn = NearestNeighbors(metric='cosine',algorithm='brute',n_neighbors=10)

In [16]:
knn.fit(matrix_movies_users)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=10)

In [17]:
def recommender(movie_name, data, model, n_recommendations ):
    model.fit(data)
    idx=process.extractOne(movie_name, df['title'])[2]
    print('Movie Selected:-',df['title'][idx], 'Index: ',idx)
    print('Searching for recommendations.....')
    distances, indices=model.kneighbors(data[idx], n_neighbors=n_recommendations)
    for i in indices:
        print(df['title'][i].where(i!=idx))

In [20]:
recommender('Toy Story', matrix_movies_users, knn, 5)

Movie Selected:- Toy Story Index:  0
Searching for recommendations.....
64                         Bio-Dome
49              Usual Suspects, The
47                       Pocahontas
45    How to Make an American Quilt
50                   Guardian Angel
Name: title, dtype: object


In [24]:
movie.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [23]:
recommender('Window of the Soul', matrix_movies_users, knn, 5)

Movie Selected:- Window of the Soul Index:  62419
Searching for recommendations.....
62064                                             Netizens
62065                                         True Fiction
62063    Takumi - A 60,000 hour story on the survival o...
62061                                            Seventeen
62066                   A Cinderella Story: Christmas Wish
Name: title, dtype: object


In [25]:
recommender('Sudden Death', matrix_movies_users, knn, 5)

Movie Selected:- Sudden Death Index:  8
Searching for recommendations.....
64                         Bio-Dome
49              Usual Suspects, The
47                       Pocahontas
45    How to Make an American Quilt
50                   Guardian Angel
Name: title, dtype: object
